In [1]:
from torch.utils.tensorboard import SummaryWriter
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision.transforms import transforms

torch.set_printoptions(linewidth=120)


In [2]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()


In [3]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):
        t = t

        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = t.reshape(-1, 12*4*4)
        t = self.fc1(t)
        t = F.relu(t)

        t = self.fc2(t)
        t = F.relu(t)
        t = self.out(t)

        return t


In [4]:
train_set = torchvision.datasets.FashionMNIST(
    root='/.data/', train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))


In [5]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True)

In [6]:
tb = SummaryWriter()

network = Network()
images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

tb.add_image('images', grid)
tb.add_graph(network, images)
tb.close()

# tensorboard - -logdir = ./runs/ ---> to run tensorboard


In [7]:
batch_size = 100
lr = 0.01

network = Network()
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
optimizer = optim.Adam(network.parameters(), lr=lr)

images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

comment = f'batch_size={batch_size}, lr={lr}'
tb = SummaryWriter(comment=comment)
tb.add_image('images', grid)
tb.add_graph(network, images)

for epoch in range(10):

    total_loss = 0
    total_correct = 0

    for batch in train_loader:  # Get Batch

        images, labels = batch
        preds = network(images)
        # Calculate Loss
        loss = F.cross_entropy(preds, labels)
        # Calculate Gradient
        optimizer.zero_grad()
        loss.backward()

        # Update Weights
        optimizer.step()

        total_loss += loss.item() * batch_size
        total_correct += get_num_correct(preds, labels)

    tb.add_scalar('Loss', total_loss, epoch)
    tb.add_scalar('Number Correct', total_correct, epoch)
    tb.add_scalar('Accuracy', total_correct / len(train_set), epoch)

    # tb.add_histogram('conv1.bias', network.conv1.bias, epoch)
    # tb.add_histogram('conv1.weight', network.conv1.weight, epoch)
    # tb.add_histogram('conv1.weight.grad', network.conv1.weight.grad, epoch)

    for name, weight in network.named_parameters():
        tb.add_histogram(name,weight, epoch)
        tb.add_histogram(f'{name}.grad', weight.grad, epoch)

    print("epoch", epoch, "total_correct:", total_correct, "loss:", total_loss)

tb.close()


epoch 0 total_correct: 47210 loss: 33439.98210132122
epoch 1 total_correct: 51517 loss: 23193.794877827168
epoch 2 total_correct: 52159 loss: 21075.26876181364
epoch 3 total_correct: 52418 loss: 20527.27068811655
epoch 4 total_correct: 52475 loss: 20289.772848784924
epoch 5 total_correct: 52867 loss: 19194.725701212883
epoch 6 total_correct: 52867 loss: 19271.2292984128
epoch 7 total_correct: 52868 loss: 18900.85325539112
epoch 8 total_correct: 52965 loss: 18633.73486995697
epoch 9 total_correct: 53148 loss: 18337.95399814844


#### tensorboard --logdir=/d:/jupyter/pytorch/runs to run tensorboard on a specified folder


# creating run builder

In [9]:
from collections import OrderedDict
from collections import namedtuple
from itertools import product

In [10]:
class RunBuilder():
    @staticmethod
    def get_runs(params):

        Run = namedtuple('Run', params.keys())

        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))

        return runs


In [11]:
params = OrderedDict(
    lr = [0.01, 0.001],
    batch_size = [1000,10000]
)

In [34]:
runs = RunBuilder.get_runs(params)
runs

[Run(lr=0.01, batch_size=1000),
 Run(lr=0.01, batch_size=10000),
 Run(lr=0.001, batch_size=1000),
 Run(lr=0.001, batch_size=10000)]

In [38]:
run = runs[0]
run

Run(lr=0.01, batch_size=1000)

In [39]:
print(run.lr, run.batch_size)

0.01 1000


In [40]:
for run in runs:
    print(run, run.lr, run.batch_size)


Run(lr=0.01, batch_size=1000) 0.01 1000
Run(lr=0.01, batch_size=10000) 0.01 10000
Run(lr=0.001, batch_size=1000) 0.001 1000
Run(lr=0.001, batch_size=10000) 0.001 10000


In [41]:
params = OrderedDict(
    lr=[.01, .001],
    batch_size=[1000, 10000]
)


In [43]:
params.keys()

odict_keys(['lr', 'batch_size'])

In [44]:
params.values()

odict_values([[0.01, 0.001], [1000, 10000]])

In [45]:
Run = namedtuple('Run', params.keys())
Run

__main__.Run

In [47]:
runs = []
for v in product(*params.values()):
    runs.append(Run(*v))

runs

[Run(lr=0.01, batch_size=1000),
 Run(lr=0.01, batch_size=10000),
 Run(lr=0.001, batch_size=1000),
 Run(lr=0.001, batch_size=10000)]

In [49]:
class RunBuilder():
    @staticmethod
    def get_runs(params):

        Run = namedtuple('Run', params.keys())

        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))

        return runs
